In [1]:
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, Dense, Flatten, Input, LayerNormalization, Add, Conv1D, MaxPooling1D, LSTM
from tensorflow.keras.models import Model, Sequential

from sklearn.metrics import classification_report, confusion_matrix

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
def load_data(name, prefix, file_format = "pkl"):
    path = prefix + f"{name}.{file_format}"
    with open(path, "rb") as f:
        return pickle.load(f)


X_train = load_data("X_train_final", prefix="../datas/demo dataset/")
X_train_flipped = load_data("X_train_final", prefix="../datas/demo dataset flipped/")
X_train = np.concatenate((X_train, X_train_flipped), axis = 0)

X_test = load_data("X_test_final", prefix="../datas/demo dataset/")
X_test_flipped = load_data("X_test_final", prefix="../datas/demo dataset flipped/")
X_test = np.concatenate((X_test, X_test_flipped), axis = 0)

X_val = load_data("X_val_final", prefix="../datas/demo dataset/")
X_val_flipped = load_data("X_val_final", prefix="../datas/demo dataset flipped/")
X_val = np.concatenate((X_val, X_val_flipped), axis = 0)


y_train = load_data("y_train_final", prefix="../datas/demo dataset/")
y_train_flipped = load_data("y_train_final", prefix="../datas/demo dataset flipped/")
y_train = np.concatenate((y_train, y_train_flipped), axis = 0)

y_test = load_data("y_test_final", prefix="../datas/demo dataset/")
y_test_flipped = load_data("y_test_final", prefix="../datas/demo dataset flipped/")
y_test = np.concatenate((y_test, y_test_flipped), axis = 0)

y_val = load_data("y_val_final", prefix="../datas/demo dataset/")
y_val_flipped = load_data("y_val_final", prefix="../datas/demo dataset flipped/")
y_val = np.concatenate((y_val, y_val_flipped), axis = 0)

## Transformer Learning

In [3]:
def transformer_encoder(inputs, head_size, num_heads, dropout_rate=0.18):
    x = layers.LayerNormalization()(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(x, x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Add()([x, inputs])
    x = layers.LayerNormalization()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    return layers.Add()([x, x])

input_layer = layers.Input(shape=(40, 86))
x = transformer_encoder(input_layer, head_size=32, num_heads=4)
x = layers.Flatten()(x)
output = layers.Dense(269, activation='softmax')(x)

transformer_model = Model(inputs=input_layer, outputs=output)
transformer_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
transformer_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 40, 86)]     0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 40, 86)      172         ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 40, 86)      44502       ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

In [4]:
transformer_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=45, batch_size=32, verbose=1)
transformer_model.evaluate(X_test, y_test)

Epoch 1/45
18/18 [==============================] - 6s 43ms/step - loss: 3.1671 - accuracy: 0.1661 - val_loss: 2.3026 - val_accuracy: 0.2337
Epoch 2/45
18/18 [==============================] - 0s 12ms/step - loss: 1.9917 - accuracy: 0.3520 - val_loss: 1.6311 - val_accuracy: 0.3859
Epoch 3/45
18/18 [==============================] - 0s 12ms/step - loss: 1.5848 - accuracy: 0.4765 - val_loss: 1.7786 - val_accuracy: 0.4783
Epoch 4/45
18/18 [==============================] - 0s 11ms/step - loss: 1.3701 - accuracy: 0.5433 - val_loss: 1.6421 - val_accuracy: 0.4457
Epoch 5/45
18/18 [==============================] - 0s 12ms/step - loss: 1.1272 - accuracy: 0.6191 - val_loss: 1.4730 - val_accuracy: 0.5000
Epoch 6/45
18/18 [==============================] - 0s 12ms/step - loss: 0.9216 - accuracy: 0.6895 - val_loss: 1.4435 - val_accuracy: 0.6141
Epoch 7/45
18/18 [==============================] - 0s 12ms/step - loss: 0.8465 - accuracy: 0.7094 - val_loss: 1.6657 - val_accuracy: 0.5109
Epoch 8/45
18

[1.070172905921936, 0.7924528121948242]

## CNN + LSTM/GRU Hybrid

In [5]:
LSTM_GRU = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(40, 86)),
    MaxPooling1D(2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(2),
    
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(64),

    layers.Dense(269, activation='softmax')
])

LSTM_GRU.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
LSTM_GRU.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=0)
LSTM_GRU.evaluate(X_test, y_test)

4/4 [==============================] - 0s 7ms/step - loss: 1.1441 - accuracy: 0.6792


[1.144097924232483, 0.6792452931404114]

## 3D Convolutional Neural Network (3D-CNN)

In [7]:
input_layer = Input(shape=(40, 86))
x = Conv1D(32, kernel_size=3, activation='relu')(input_layer)
x = MaxPooling1D(2)(x)
x = Conv1D(64, kernel_size=3, activation='relu')(x)
x = MaxPooling1D(2)(x)

x = Flatten()(x)
output = Dense(269, activation='softmax')(x)

CNN_3D = Model(inputs=input_layer, outputs=output)
CNN_3D.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
CNN_3D.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=0)
CNN_3D.evaluate(X_test, y_test)

4/4 [==============================] - 0s 7ms/step - loss: 1.5915 - accuracy: 0.7642


[1.5915217399597168, 0.7641509175300598]

##  Recurrent CNN (RCNN)

In [9]:
input_layer = Input(shape=(40, 86))
x = Conv1D(64, kernel_size=3, activation='relu')(input_layer)
x = Conv1D(128, kernel_size=3, activation='relu')(x)

x = LSTM(128, return_sequences=True)(x)
x = LSTM(64)(x)

output = Dense(269, activation='softmax')(x)

RCNN = Model(inputs=input_layer, outputs=output)
RCNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
RCNN.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=0)
RCNN.evaluate(X_test, y_test)

4/4 [==============================] - 0s 7ms/step - loss: 1.0171 - accuracy: 0.6604


[1.0171003341674805, 0.6603773832321167]